In [1]:
import json
import pandas as pd
from pyspark.sql import SparkSession
import glob
import os

In [ ]:
df = pd.DataFrame()

# Loop through years from 2018 to 2023
for year in range(2018, 2024):
    directory = f"Project/{year}/"

    # Ensure the directory exists before proceeding
    if not os.path.exists(directory):
        print(f"Directory {directory} does not exist.")
        continue

    # Get all JSON files in the directory
    json_files = [f for f in os.listdir(directory) if f.endswith(".json")]

    # Sort the files to ensure they are processed in order
    json_files.sort()

    for file_name in json_files:
        file_path = os.path.join(directory, file_name)
        print("Reading", file_path)

        if os.path.exists(file_path):
            try:
                with open(file_path, "r", encoding="utf-8") as file:
                    data = json.load(file)

                temp_df = pd.json_normalize(data)
                df = pd.concat([df, temp_df], ignore_index=True)
            except UnicodeDecodeError as e:
                print(f"UnicodeDecodeError while processing {file_path}: {e}")
            except json.JSONDecodeError as e:
                print(f"JSONDecodeError in file {file_path}: {e}")
        else:
            print(f"File {file_path} does not exist.")
            
original = df.copy()

Reading Project/2018/201800000.json
Reading Project/2018/201800001.json
Reading Project/2018/201800002.json
Reading Project/2018/201800003.json
Reading Project/2018/201800004.json
Reading Project/2018/201800005.json
Reading Project/2018/201800006.json
Reading Project/2018/201800007.json
Reading Project/2018/201800008.json
Reading Project/2018/201800009.json
Reading Project/2018/201800010.json
Reading Project/2018/201800011.json
Reading Project/2018/201800012.json
Reading Project/2018/201800013.json
Reading Project/2018/201800014.json
Reading Project/2018/201800015.json
Reading Project/2018/201800016.json
Reading Project/2018/201800017.json
Reading Project/2018/201800018.json
Reading Project/2018/201800019.json
Reading Project/2018/201800020.json
Reading Project/2018/201800021.json
Reading Project/2018/201800022.json
Reading Project/2018/201800023.json
Reading Project/2018/201800024.json
Reading Project/2018/201800025.json
Reading Project/2018/201800026.json
Reading Project/2018/2018000

In [4]:
df = original.copy()
# df = pandas_df.copy()

In [5]:
df.columns

Index(['abstracts-retrieval-response.item.ait:process-info.ait:status.@state',
       'abstracts-retrieval-response.item.ait:process-info.ait:status.@type',
       'abstracts-retrieval-response.item.ait:process-info.ait:status.@stage',
       'abstracts-retrieval-response.item.ait:process-info.ait:date-delivered.@day',
       'abstracts-retrieval-response.item.ait:process-info.ait:date-delivered.@timestamp',
       'abstracts-retrieval-response.item.ait:process-info.ait:date-delivered.@year',
       'abstracts-retrieval-response.item.ait:process-info.ait:date-delivered.@month',
       'abstracts-retrieval-response.item.ait:process-info.ait:date-sort.@day',
       'abstracts-retrieval-response.item.ait:process-info.ait:date-sort.@year',
       'abstracts-retrieval-response.item.ait:process-info.ait:date-sort.@month',
       ...
       'abstracts-retrieval-response.item.bibrecord.head.enhancement.descriptorgroup.descriptors.@type',
       'abstracts-retrieval-response.item.bibrecord.head

In [6]:
# Remove null columns
threshold = 0.5

max_null = 0

for col in df.columns:
    n_null = df[col].isnull().mean()

    if n_null > max_null:
        max_null = n_null
    
    if df[col].isnull().mean() > threshold:
        df.drop(columns=col, axis=1, inplace=True)
        
print(max_null)

1.0


In [7]:
df.columns

Index(['abstracts-retrieval-response.item.ait:process-info.ait:status.@state',
       'abstracts-retrieval-response.item.ait:process-info.ait:status.@type',
       'abstracts-retrieval-response.item.ait:process-info.ait:status.@stage',
       'abstracts-retrieval-response.item.ait:process-info.ait:date-delivered.@day',
       'abstracts-retrieval-response.item.ait:process-info.ait:date-delivered.@timestamp',
       'abstracts-retrieval-response.item.ait:process-info.ait:date-delivered.@year',
       'abstracts-retrieval-response.item.ait:process-info.ait:date-delivered.@month',
       'abstracts-retrieval-response.item.ait:process-info.ait:date-sort.@day',
       'abstracts-retrieval-response.item.ait:process-info.ait:date-sort.@year',
       'abstracts-retrieval-response.item.ait:process-info.ait:date-sort.@month',
       ...
       'abstracts-retrieval-response.item.xocs:meta.xocs:funding-list.xocs:funding',
       'abstracts-retrieval-response.item.xocs:meta.xocs:funding-list.xocs:f

In [8]:
# Remove useless columns

useless_cols = [
    "abstracts-retrieval-response.item.ait:process-info.ait:status.@type",
    "abstracts-retrieval-response.item.ait:process-info.ait:status.@state",
    "abstracts-retrieval-response.item.ait:process-info.ait:status.@stage",  # just status
    "abstracts-retrieval-response.item.ait:process-info.ait:date-delivered.@day",
    "abstracts-retrieval-response.item.ait:process-info.ait:date-delivered.@timestamp",
    "abstracts-retrieval-response.item.ait:process-info.ait:date-delivered.@month",  # just year is enough
    "abstracts-retrieval-response.item.ait:process-info.ait:date-sort.@day",
    "abstracts-retrieval-response.item.ait:process-info.ait:date-sort.@month",
    "abstracts-retrieval-response.item.bibrecord.head.abstracts",
    "abstracts-retrieval-response.item.bibrecord.head.correspondence.affiliation.@country",  # is short form of country
    "abstracts-retrieval-response.item.bibrecord.head.citation-info.citation-type.@code",  # just type
    "abstracts-retrieval-response.item.bibrecord.head.citation-info.citation-language.@xml:lang",  # short form of language
    "abstracts-retrieval-response.item.bibrecord.head.source.sourcetitle-abbrev",  # title not needed
    "abstracts-retrieval-response.item.bibrecord.head.source.website.ce:e-address.$",  # website
    "abstracts-retrieval-response.item.bibrecord.head.source.website.ce:e-address.@type",  # type of above
    "abstracts-retrieval-response.item.bibrecord.head.source.volisspag.pagerange.@first",
    "abstracts-retrieval-response.item.bibrecord.head.source.volisspag.pagerange.@last",  # could be good if we do how much pages ref prediction
    "abstracts-retrieval-response.item.bibrecord.head.source.@type",  # just type
    "abstracts-retrieval-response.item.bibrecord.head.source.sourcetitle",  # just title
    "abstracts-retrieval-response.item.bibrecord.head.source.@srcid",  # id
    "abstracts-retrieval-response.item.bibrecord.head.source.publicationdate.month",
    "abstracts-retrieval-response.item.bibrecord.head.source.publicationdate.year",  # redundant publish year
    "abstracts-retrieval-response.item.bibrecord.head.source.publicationdate.date-text.@xfab-added",
    "abstracts-retrieval-response.item.bibrecord.head.source.publicationdate.date-text.$",
    "abstracts-retrieval-response.item.bibrecord.head.source.publicationdate.day",  # just year is enough
    "abstracts-retrieval-response.item.bibrecord.head.correspondence.person.ce:initials", # useless first char name
    "abstracts-retrieval-response.item.bibrecord.item-info.copyright.$",
    "abstracts-retrieval-response.item.bibrecord.item-info.copyright.@type",
    "abstracts-retrieval-response.item.bibrecord.item-info.history.date-created.@day",
    "abstracts-retrieval-response.item.bibrecord.item-info.history.date-created.@timestamp",
    "abstracts-retrieval-response.item.bibrecord.item-info.history.date-created.@year",
    "abstracts-retrieval-response.item.bibrecord.item-info.history.date-created.@month",  # why it created in 2020 when it is 2018 file 💀
    "abstracts-retrieval-response.item.bibrecord.item-info.itemidlist.itemid",
    "abstracts-retrieval-response.item.bibrecord.item-info.itemidlist.ce:doi",  # ids
    "abstracts-retrieval-response.coredata.srctype",  # just type
    "abstracts-retrieval-response.coredata.eid",  # ids
    "abstracts-retrieval-response.coredata.prism:url",
    "abstracts-retrieval-response.coredata.subtypeDescription",
    "abstracts-retrieval-response.coredata.link",
    "abstracts-retrieval-response.coredata.source-id",
    "abstracts-retrieval-response.coredata.prism:endingPage",
    "abstracts-retrieval-response.coredata.openaccess",
    "abstracts-retrieval-response.coredata.openaccessFlag",  # not necessary
    "abstracts-retrieval-response.coredata.prism:doi",
    "abstracts-retrieval-response.coredata.prism:startingPage",
    "abstracts-retrieval-response.coredata.subtype",
    "abstracts-retrieval-response.coredata.dc:identifier",  # also id
    "abstracts-retrieval-response.coredata.publishercopyright",
    # "abstracts-retrieval-response.coredata.dc:publisher",
    "abstracts-retrieval-response.language.@xml:lang",
    "abstracts-retrieval-response.authors.author",  # redundant author
    "abstracts-retrieval-response.item.xocs:meta.xocs:funding-list.@pui-match",  # what is this?
    "abstracts-retrieval-response.item.xocs:meta.xocs:funding-list.@has-funding-info",
    "abstracts-retrieval-response.item.xocs:meta.xocs:funding-list.xocs:funding-addon-generated-timestamp",
    "abstracts-retrieval-response.item.xocs:meta.xocs:funding-list.xocs:funding-addon-type",  # link
    "abstracts-retrieval-response.item.bibrecord.head.citation-info.abstract-language.@xml:lang",  # redundant with language
    "abstracts-retrieval-response.item.bibrecord.head.source.translated-sourcetitle.$", # title
    "abstracts-retrieval-response.item.bibrecord.head.source.volisspag.voliss.@volume",
    "abstracts-retrieval-response.item.bibrecord.head.source.issn",  # id?
    "abstracts-retrieval-response.coredata.dc:description",
    "abstracts-retrieval-response.coredata.prism:volume",
    "abstracts-retrieval-response.coredata.prism:issn",
    "abstracts-retrieval-response.item.xocs:meta.xocs:funding-list.xocs:funding-text",  # basically desc
    "abstracts-retrieval-response.item.bibrecord.head.correspondence.affiliation.postal-code",
    "abstracts-retrieval-response.item.bibrecord.head.citation-info.author-keywords.author-keyword",  # probably too many unique
    "abstracts-retrieval-response.authkeywords.author-keyword",  # probably too many unique
    "abstracts-retrieval-response.item.bibrecord.head.source.volisspag.voliss.@issue",
    "abstracts-retrieval-response.coredata.prism:issueIdentifier",
    "abstracts-retrieval-response.coredata.prism:pageRange",
    "abstracts-retrieval-response.item.xocs:meta.xocs:funding-list.xocs:funding", # we donot care funding
    "abstracts-retrieval-response.item.bibrecord.head.grantlist.@complete",
    "abstracts-retrieval-response.item.bibrecord.head.grantlist.grant-text.$",
    "abstracts-retrieval-response.item.bibrecord.head.grantlist.grant-text.@xml:lang",
    "abstracts-retrieval-response.idxterms.mainterm",
    "abstracts-retrieval-response.item.bibrecord.head.source.translated-sourcetitle.@xml:lang",
]

df.drop(columns=useless_cols, axis=1, inplace=True)

In [9]:
df.columns

Index(['abstracts-retrieval-response.item.ait:process-info.ait:date-delivered.@year',
       'abstracts-retrieval-response.item.ait:process-info.ait:date-sort.@year',
       'abstracts-retrieval-response.item.bibrecord.head.author-group',
       'abstracts-retrieval-response.item.bibrecord.head.citation-title',
       'abstracts-retrieval-response.item.bibrecord.head.correspondence.affiliation.country',
       'abstracts-retrieval-response.item.bibrecord.head.correspondence.affiliation.city',
       'abstracts-retrieval-response.item.bibrecord.head.correspondence.affiliation.organization',
       'abstracts-retrieval-response.item.bibrecord.head.correspondence.person.ce:given-name',
       'abstracts-retrieval-response.item.bibrecord.head.correspondence.person.ce:surname',
       'abstracts-retrieval-response.item.bibrecord.head.correspondence.person.ce:indexed-name',
       'abstracts-retrieval-response.item.bibrecord.head.citation-info.citation-language.@language',
       'abstracts-

In [10]:
column_renaming = {
    "abstracts-retrieval-response.item.ait:process-info.ait:date-delivered.@year": "date_delivered_year",
    "abstracts-retrieval-response.item.ait:process-info.ait:date-sort.@year": "date_sort_year",
    "abstracts-retrieval-response.item.bibrecord.head.author-group": "author_group",
    "abstracts-retrieval-response.item.bibrecord.head.citation-title": "citation_title",
    # "abstracts-retrieval-response.item.bibrecord.head.abstracts": "abstracts",
    "abstracts-retrieval-response.item.bibrecord.head.correspondence.affiliation.country": "affiliation_country",
    "abstracts-retrieval-response.item.bibrecord.head.correspondence.affiliation.city": "affiliation_city",
    "abstracts-retrieval-response.item.bibrecord.head.correspondence.affiliation.organization": "affiliation_organization",
    "abstracts-retrieval-response.item.bibrecord.head.correspondence.person.ce:given-name": "corresponding_author_given_name",
    # "abstracts-retrieval-response.item.bibrecord.head.correspondence.person.ce:initials": "corresponding_author_initials",
    "abstracts-retrieval-response.item.bibrecord.head.correspondence.person.ce:surname": "corresponding_author_surname",
    "abstracts-retrieval-response.item.bibrecord.head.correspondence.person.ce:indexed-name": "corresponding_author_indexed_name",
    "abstracts-retrieval-response.item.bibrecord.head.citation-info.citation-language.@language": "citation_language",
    "abstracts-retrieval-response.item.bibrecord.head.citation-info.abstract-language.@language": "abstract_language",
    "abstracts-retrieval-response.item.bibrecord.head.source.@country": "source_country",
    # "abstracts-retrieval-response.item.bibrecord.head.source.translated-sourcetitle.@xml:lang": "source_translated_title_lang",
    "abstracts-retrieval-response.item.bibrecord.head.source.publicationyear.@first": "source_publication_year",
    "abstracts-retrieval-response.item.bibrecord.head.source.publisher.publishername": "source_publisher_name",
    "abstracts-retrieval-response.item.bibrecord.head.enhancement.classificationgroup.classifications": "classificationgroup",
    "abstracts-retrieval-response.item.bibrecord.item-info.dbcollection": "dbcollection",
    "abstracts-retrieval-response.item.bibrecord.tail.bibliography.@refcount": "ref_count",
    "abstracts-retrieval-response.item.bibrecord.tail.bibliography.reference": "reference",
    "abstracts-retrieval-response.affiliation": "affiliation",
    "abstracts-retrieval-response.coredata.prism:coverDate": "coverDate",
    "abstracts-retrieval-response.coredata.prism:aggregationType": "aggregationType",
    "abstracts-retrieval-response.coredata.dc:creator.author": "author",
    "abstracts-retrieval-response.coredata.prism:publicationName": "publicationName",
    "abstracts-retrieval-response.coredata.citedby-count": "citedby_count",
    # "abstracts-retrieval-response.coredata.subtype": "subtype",
    # "abstracts-retrieval-response.coredata.prism:pageRange": "pageRange",
    "abstracts-retrieval-response.coredata.dc:title": "title",
    "abstracts-retrieval-response.subject-areas.subject-area": "subject_area",
    # "abstracts-retrieval-response.coredata.publishercopyright": "publishercopyright",
    "abstracts-retrieval-response.coredata.dc:publisher": "publisher",
    # "abstracts-retrieval-response.idxterms.mainterm": "mainterm",
}

df.rename(columns=column_renaming, inplace=True)

In [11]:
# df.dropna(axis=0, inplace=True)

for col in df.columns:
    # if (df[col].isnull()):
    print(col, df[col].isnull().mean())

date_delivered_year 0.0
date_sort_year 0.0
author_group 0.17620020429009192
citation_title 0.00017024174327545113
affiliation_country 0.17960503915560094
affiliation_city 0.23289070480081717
affiliation_organization 0.2257405515832482
corresponding_author_given_name 0.24021109976166155
corresponding_author_surname 0.17926455566905006
corresponding_author_indexed_name 0.17926455566905006
citation_language 0.00017024174327545113
source_country 0.00034048348655090226
source_publication_year 0.0
source_publisher_name 0.0015321756894790602
classificationgroup 0.0
dbcollection 0.0
ref_count 0.020599250936329586
reference 0.021961184882533197
affiliation 0.28106911814776986
coverDate 0.0
aggregationType 0.0
author 0.0
publicationName 0.0
citedby_count 0.0
title 0.00017024174327545113
publisher 0.0015321756894790602
subject_area 0.0
abstract_language 0.054307116104868915


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5874 entries, 0 to 5873
Data columns (total 28 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   date_delivered_year                5874 non-null   object
 1   date_sort_year                     5874 non-null   object
 2   author_group                       4839 non-null   object
 3   citation_title                     5873 non-null   object
 4   affiliation_country                4819 non-null   object
 5   affiliation_city                   4506 non-null   object
 6   affiliation_organization           4548 non-null   object
 7   corresponding_author_given_name    4463 non-null   object
 8   corresponding_author_surname       4821 non-null   object
 9   corresponding_author_indexed_name  4821 non-null   object
 10  citation_language                  5873 non-null   object
 11  source_country                     5872 non-null   object
 12  source

In [13]:
# important col
df['subject_area'] = df['subject_area'].fillna('Unknown')

In [14]:
# convert to datetime
df['coverDate'] = pd.to_datetime(df['coverDate'], errors='coerce')

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5874 entries, 0 to 5873
Data columns (total 28 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   date_delivered_year                5874 non-null   object        
 1   date_sort_year                     5874 non-null   object        
 2   author_group                       4839 non-null   object        
 3   citation_title                     5873 non-null   object        
 4   affiliation_country                4819 non-null   object        
 5   affiliation_city                   4506 non-null   object        
 6   affiliation_organization           4548 non-null   object        
 7   corresponding_author_given_name    4463 non-null   object        
 8   corresponding_author_surname       4821 non-null   object        
 9   corresponding_author_indexed_name  4821 non-null   object        
 10  citation_language                  5

In [16]:
# Convert columns to numeric
df['source_publication_year'] = pd.to_numeric(df['source_publication_year'], errors='coerce')
df['date_sort_year'] = pd.to_numeric(df['date_sort_year'], errors='coerce')
df['date_delivered_year'] = pd.to_numeric(df['date_delivered_year'], errors='coerce')

df['citedby_count'] = pd.to_numeric(df['citedby_count'], errors='coerce')
df['ref_count'] = pd.to_numeric(df['ref_count'], errors='coerce')

# Fill NaN with the mean value and convert to integer type
df['citedby_count'] = df['citedby_count'].fillna(df['citedby_count'].mean()).astype(int)
df['ref_count'] = df['ref_count'].fillna(df['ref_count'].mean()).astype(int)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5874 entries, 0 to 5873
Data columns (total 28 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   date_delivered_year                5874 non-null   int64         
 1   date_sort_year                     5874 non-null   int64         
 2   author_group                       4839 non-null   object        
 3   citation_title                     5873 non-null   object        
 4   affiliation_country                4819 non-null   object        
 5   affiliation_city                   4506 non-null   object        
 6   affiliation_organization           4548 non-null   object        
 7   corresponding_author_given_name    4463 non-null   object        
 8   corresponding_author_surname       4821 non-null   object        
 9   corresponding_author_indexed_name  4821 non-null   object        
 10  citation_language                  5

In [18]:
df.fillna("Unknown", inplace=True)

In [ ]:
pd.set_option('display.max_columns', None)  # Show all columns
df

,date_delivered_year,date_sort_year,author_group,citation_title,affiliation_country,affiliation_city,affiliation_organization,corresponding_author_given_name,corresponding_author_surname,corresponding_author_indexed_name,citation_language,source_country,source_publication_year,source_publisher_name,classificationgroup,dbcollection,ref_count,reference,affiliation,coverDate,aggregationType,author,publicationName,citedby_count,title,publisher,subject_area,abstract_language
0,2020,2018,"[{'affiliation': {'country': 'Thailand', '@afi...",Public health and international epidemiology f...,Thailand,Bangkok,[{'$': 'Department of Preventive and Social Me...,Krit,Pongpirul,Pongpirul K.,English,usa,2018,Springer International Publishing,"[{'@type': 'ASJC', 'classification': '2700'}, ...","[{'$': 'SCOPUS'}, {'$': 'SNBOOK'}, {'$': 'Scop...",76,[{'ref-fulltext': 'Winslow CE. The untilled fi...,"[{'affiliation-city': 'Stanford', '@id': '6003...",2018-12-31,Book,"[{'ce:given-name': 'Krit', 'preferred-name': {...","Radiology in Global Health: Strategies, Implem...",1,Public health and international epidemiology f...,Springer International Publishing,"[{'@_fa': 'true', '$': 'Medicine (all)', '@cod...",Unknown
1,2020,2018,Unknown,Flexible Printed Active Antenna for Digital Te...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,English,usa,2018,Institute of Electrical and Electronics Engine...,"[{'@type': 'ASJC', 'classification': [{'$': '2...","[{'$': 'CPX'}, {'$': 'REAXYSCAR'}, {'$': 'SCOP...",4,"[{'ref-fulltext': 'Pratumsiri, T. And P. Janpu...",Unknown,2018-12-31,Conference Proceeding,"[{'ce:given-name': 'Teerapong', 'preferred-nam...",Progress in Electromagnetics Research Symposium,1,Flexible Printed Active Antenna for Digital Te...,Institute of Electrical and Electronics Engine...,"[{'@_fa': 'true', '$': 'Electrical and Electro...",English
2,2021,2018,"[{'affiliation': {'country': 'Thailand', 'post...",Parametric study of hydrogen production via so...,Thailand,Bangkok,"[{'$': 'Fuels Research Center'}, {'$': 'Depart...",Benjapon,Chalermsinsuwan,Chalermsinsuwan B.,English,gbr,2018,Elsevier Ltd,"[{'@type': 'CPXCLASS', 'classification': [{'cl...","[{'$': 'APILIT'}, {'$': 'CHEM'}, {'$': 'CPX'},...",42,"[{'ref-fulltext': 'Abanades, J.C., Anthony, E....",Unknown,2018-12-31,Journal,"[{'ce:given-name': 'Kiattikhoon', 'preferred-n...",Chemical Engineering Science,21,Parametric study of hydrogen production via so...,Elsevier Ltd,"[{'@_fa': 'true', '$': 'Chemistry (all)', '@co...",English
3,2021,2018,"[{'affiliation': {'country': 'Thailand', 'post...",Superhydrophobic coating from fluoroalkylsilan...,Thailand,Pathumthani,"[{'$': 'Faculty of Science and Technology'}, {...",Suwadee,Kongparakul,Kongparakul S.,English,nld,2018,Elsevier B.V.,"[{'@type': 'CPXCLASS', 'classification': [{'cl...","[{'$': 'CHEM'}, {'$': 'CPX'}, {'$': 'REAXYSCAR...",45,"[{'ref-fulltext': 'Abadi, S.R.H., Sebzari, M.R...","[{'affiliation-city': 'Hirosaki', '@id': '6003...",2018-12-31,Journal,"[{'ce:given-name': 'Jittraporn', 'preferred-na...",Applied Surface Science,37,Superhydrophobic coating from fluoroalkylsilan...,Elsevier B.V.,"[{'@_fa': 'true', '$': 'Chemistry (all)', '@co...",English
4,2020,2018,"[{'affiliation': {'country': 'Thailand', 'addr...",Electrochemical impedance-based DNA sensor usi...,United States,Fort Collins,"[{'$': 'Department of Chemistry'}, {'$': 'Colo...",Charles S.,Henry,Henry C.S.,English,nld,2018,Elsevier B.V.,"[{'@type': 'EMCLASS', 'classification': {'clas...","[{'$': 'CHEM'}, {'$': 'CPX'}, {'$': 'EMBASE'},...",55,"[{'ref-fulltext': 'Davies, P.D.O., Pai, M., Th...","[{'affiliation-city': 'Bangkok', '@id': '60028...",2018-12-31,Journal,"[{'ce:given-name': 'Prinjaporn', 'preferred-na...",Analytica Chimica Acta,68,Electrochemical impedance-based DNA sensor usi...,Elsevier B.V.,"[{'@_fa': 'true', '$': 'Analytical Chemistry',...",English
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5869,202